## 1. Загрузка данных

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('../datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


У нас имеется таблица с 4 признаками и одним целевым признаком ("Страховые выплаты"). Проверим данные на адеквастность и наличие аномальных значений. 

In [2]:
data.info(),\
data.describe(),\
data['Страховые выплаты'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


(None,
                Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
 count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
 mean      0.499000    30.952800  39916.360000     1.194200           0.148000
 std       0.500049     8.440807   9900.083569     1.091387           0.463183
 min       0.000000    18.000000   5300.000000     0.000000           0.000000
 25%       0.000000    24.000000  33300.000000     0.000000           0.000000
 50%       0.000000    30.000000  40200.000000     1.000000           0.000000
 75%       1.000000    37.000000  46600.000000     2.000000           0.000000
 max       1.000000    65.000000  79000.000000     6.000000           5.000000,
 0    4436
 1     423
 2     115
 3      18
 4       7
 5       1
 Name: Страховые выплаты, dtype: int64)

- Пропуски отсутствуют
- Значения в столбцах не имеют аномальных значений. 

## 2. Умножение матриц

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
- Изменится
- Не изменится

**Ответ:** 
Не изменится



In [3]:
import numpy as np

# Матрица признаков
X = np.array([
    [4, 7],
    [8, 6],
    [3, 3]
], dtype=np.float)

# Матрица таргетов
y = np.array([1, 2, 3], dtype=np.float)

# Обратимая матрица
e = np.array([
    [10, 0],
    [0, 1]
], dtype=np.float)

# Проверка

try:
    np.linalg.inv(e)
    print(f"Обратимая\n")
except:
    print("Не обратимая")

# Без регуляризации
clambda = 0.00

print(clambda * np.eye(X.shape[1]))

H = X.T @ X + clambda * np.eye(X.shape[1])
g = X.T @ y

weights = np.linalg.solve(H, g)
error = np.linalg.norm(X @ weights - y)
print()
print(f"Веса: {weights}")
print(f"Ошибка: {error}")

# Умножаем признаки на обратимую матрицу
X = X @ e

H = X.T @ X + clambda * np.eye(X.shape[1])
g = X.T @ y
weights = np.linalg.solve(H, g)
error = np.linalg.norm(X @ weights - y)
print()
print(f"Веса: {weights}")
print(f"Транспонированные веса @ обратимая матрица: {weights.T @ e}")
print(f"Ошибка: {error}")

Обратимая

[[0. 0.]
 [0. 0.]]

Веса: [0.30324277 0.02366345]
Ошибка: 2.131521287102413

Веса: [0.03032428 0.02366345]
Транспонированные веса @ обратимая матрица: [0.30324277 0.02366345]
Ошибка: 2.1315212871024127


**Обоснование**:

$a = Xw$

$w = (X^T X)^{-1}X^T y$

Умножим признаки на матрицу $E$ (Обратимая) и получим итоговую формулу:

$a = XE(EX^T XE)^{-1}EX^T y$ 



В данной формуле возможно заменить произведение $E^{-1}$ к $E$ на тождественную матрицу, умножение на которую не влияет. 

## 3. Алгоритм преобразования

**Алгоритм**

1. Переименование столбцов в цифровые значения
2. Применить к категориальным признакам OHE
3. Умножение на обратимую матрицу
4. Scaling значений


**Обоснование**

1. В данный момент столбцы прямо сообщают о персональной информации. Изменение на цифровые значения нам не позволят определить признаки. 
2. У нас имеются признаки, которые можно привести к категориальными (до 10 вариантов), однако такие функции не всегда вписываются в модель регрессии. OHE позволит перевести такие признаки в несколько бинарных, наличие / отсутствие которых позволят их вписать в линейную регрессию.
3. Умножим матрицу признаков на обратимую (исходя из вывода выше). 
4. Так как умножение на матрицу уже изменило матрицу, то мы можем провести скалирование значений. 


## 4. Проверка алгоритма

In [4]:
# Разделим наши признаки 

X = data.drop ('Страховые выплаты', axis = 1)
y = data['Страховые выплаты']

In [5]:
# 1. Переименование
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

# 1. Переименование

class Transformation:

    def __init__(self):
        self.features = X
        self.targets = y

    def rename(self, df):
        new_df = df
        new_df.columns = range(len(new_df.columns))
        return new_df
    
    def ohe(self, df, ft = [0,3]):
        onehotencoder = OneHotEncoder()
        x = onehotencoder.fit_transform(df).toarray()
        return pd.DataFrame(x)
    
    def calc(self, df):
        E = np.random.rand(df.values.shape[1], df.values.shape[1])
        # Проверка на обратимость

        try:
            np.linalg.inv(E)
        except:
            print('Не обратимая')
            
        return df @ E


In [6]:
t = Transformation()
df = t.rename(data)

X = df.drop (4, axis = 1)
y = df[4]

X_ohe = t.ohe(X)


X_final = t.calc(X_ohe)

In [7]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

def final_calc(X,X_final, y):
    
    Xs = [X, X_final]
    
    results = []
    for x in Xs:
        model = LinearRegression()
        model.fit(x,y)
        pred = model.predict(x)
        results.append(r2_score(y,pred))
        
    print("------Результаты-------")
    print(f"R2 score: \t\nИзначальные данные: {results[0]}")
    print(f"Обезличенные данные: {results[1]}")
    print(f"Разница: {np.abs(results[0] - results[1])}")
    
final_calc(X,X_final,y)

------Результаты-------
R2 score: 	
Изначальные данные: 0.42494550286668
Обезличенные данные: 0.9759743271630931
Разница: 0.5510288242964131


Наши данные обезличены и нам удалось достичь меньшей ошибки 